In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [ ]:
# from google.colab import files
# files.os.listdir('/gdrive/My Drive/')
import tarfile
tf = tarfile.open('/gdrive/My Drive/datasets/EnglishFnt.tgz')

In [ ]:
tf.extractall()

In [ ]:
import numpy as np
from skimage import io, transform
from skimage.color import rgb2gray
from skimage.transform import resize
from random import shuffle

X_train=[]
X_test=[]
Y_train=[]
Y_test=[]
#read images
for i in range(1,63):
  y = np.zeros(62)
  y[i-1] = 1
  s="%03d"%i
  directory="./English/Fnt/Sample"+s
  index=np.arange(1,1017)
  shuffle(index)
  for j in range(0,1016):
    s2="%05d"%index[j]
    img_name=directory+"/img"+s+"-"+s2+".png"
    img=io.imread(img_name)
    img = rgb2gray(img)
    img = resize(img, (15,19))
    if j<int(0.9*1016): #split into training, val, and test
      X_train.append(img)
      Y_train.append(y)
    else:
      X_test.append(img)
      Y_test.append(y)


In [ ]:
X_train=np.asarray(X_train)/255
X_train = X_train.reshape((X_train.shape[0], 15, 19, 1))
X_test=np.asarray(X_test)/255
X_test = X_test.reshape((X_test.shape[0], 15, 19, 1))
Y_train=np.asarray(Y_train)
Y_test=np.asarray(Y_test)

print("Training dataset ", X_train.shape, Y_train.shape)

Training dataset  (56668, 15, 19, 1) (56668, 62)


In [ ]:
len(X_train[0])

56668

In [ ]:
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, Dense, Activation, ZeroPadding2D, Flatten, Conv2D, MaxPooling2D, Dropout
from keras import regularizers
import keras
from keras.callbacks import EarlyStopping
from keras import regularizers

# model = Sequential()
# model.add(Conv2D(8, kernel_size=(3, 3), padding = 'valid', strides = (1,1), input_shape=(15, 19, 1), kernel_regularizer =regularizers.l2(0.001) ))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv2D(8, kernel_size=(3, 3), padding = 'same', strides = (1,1), kernel_regularizer=regularizers.l2(0.01)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv2D(16, kernel_size=(3, 3), padding = 'same', strides = (2,2), kernel_regularizer=regularizers.l2(0.01)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv2D(16, kernel_size=(3, 3), padding = 'same', strides = (1,1), kernel_regularizer=regularizers.l2(0.01)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv2D(16, kernel_size=(3, 3), padding = 'same', strides = (1,1), kernel_regularizer=regularizers.l2(0.01)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Conv2D(32, kernel_size=(3, 3), padding = 'same', strides = (2,2), kernel_regularizer=regularizers.l2(0.01)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(Conv2D(32, kernel_size=(3, 3), padding = 'same', strides = (1,1), kernel_regularizer=regularizers.l2(0.01)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(Conv2D(32, kernel_size=(3, 3), padding = 'same', strides = (1,1), kernel_regularizer=regularizers.l2(0.01)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(Conv2D(8, kernel_size=(3, 3), padding = 'same', strides = (1,1), kernel_regularizer=regularizers.l2(0.01)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(Flatten())
# model.add(Dense(62, activation='softmax' ))

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=tuple(X_train.shape[1:])))
model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
# model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01))) #maybe suppress this layer and instead train for batch size = 64, epochs = 20
# model.add(BatchNormalization())
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
# model.add(Dropout(0.5))
model.add(Dense(62, activation='softmax'))

epochs = 30
batch_size = 64

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.5)
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Epoch 1/30
886/886 - 9s - loss: 2.5259 - accuracy: 0.7656
Epoch 2/30
886/886 - 9s - loss: 1.0742 - accuracy: 0.8085
Epoch 3/30
886/886 - 9s - loss: 0.9418 - accuracy: 0.8201
Epoch 4/30
886/886 - 9s - loss: 0.8799 - accuracy: 0.8270
Epoch 5/30
886/886 - 9s - loss: 0.8433 - accuracy: 0.8314
Epoch 6/30
886/886 - 9s - loss: 0.8035 - accuracy: 0.8372
Epoch 7/30
886/886 - 9s - loss: 0.7761 - accuracy: 0.8386
Epoch 8/30
886/886 - 9s - loss: 0.7571 - accuracy: 0.8424
Epoch 9/30
886/886 - 9s - loss: 0.7195 - accuracy: 0.8462
Epoch 10/30
886/886 - 9s - loss: 0.6958 - accuracy: 0.8468
Epoch 11/30
886/886 - 9s - loss: 0.6719 - accuracy: 0.8500
Epoch 12/30
886/886 - 9s - loss: 0.6418 - accuracy: 0.8530
Epoch 13/30
886/886 - 9s - loss: 0.6306 - accuracy: 0.8550
Epoch 14/30
886/886 - 9s - loss: 0.6192 - accuracy: 0.8555
Epoch 15/30
886/886 - 9s - loss: 0.6045 - accuracy: 0.8550
Epoch 16/30
886/886 - 9s - loss: 0.6003 - accuracy: 0.8580
Epoch 17/30
886/886 - 9s - loss: 0.5908 - accuracy: 0.8568
Epoch 

In [ ]:
top1 = 0.0
top5 = 0.0    
class_probs = model.predict(X_test)
for i, l in enumerate(Y_test):
  label = np.where(l == 1)
  class_prob = class_probs[i]
  top_values = (-class_prob).argsort()[:3]
  if top_values[0] in label:
    top1 += 1.0
  if np.isin(np.array(label), top_values):
    top5 += 1.0

print("top1 acc", top1/len(Y_test))
print("top1 acc", top5/len(Y_test))

top1 acc 0.8347564832384566
top1 acc 0.9716951296647691


In [ ]:
model.save('classifierpaper_EnglishFNT_0-1.h5')
model.count_params()

12231486

In [ ]:
model2 = keras.models.load_model('classifierpaper_EnglishFNT.h5')
model2.count_params()

56958